# Random Forest Classification with XG BOOST and K-Fold Cross Validation


## Importing the libraries, clean data and convert day time to only month


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from matplotlib import dates as mpl_dates

In [ ]:
personal_dir = '/content/drive/Shareddrives/arv_msg/colab_notebooks/'

**clean*



In [ ]:
df= pd.read_csv(personal_dir+'data_nocloud.csv')
df2= pd.read_csv(personal_dir+'data_2_nocloud.csv')
df.sort_values(by=['date','label'],ascending= True, inplace= True), df2.sort_values(by=['date','label'],ascending= True, inplace= True)
df=df.reset_index().iloc[:,2:-1]
df2=df2.reset_index().iloc[:,3:]
df_all= pd.concat([df,df2],axis=1)
pd.set_option('use_inf_as_na', True)
df_all=df_all[df_all.ndvi<=1].dropna().drop('ndre', axis=1)


In [ ]:
df_all.head(20)


Divide the date into seperate indices

In [ ]:
int(df.iloc[0,0][8:])

In [ ]:
#split monthly index
for k in range(df_all.shape[0]):
  if int(df_all.iloc[k,0][8:])>15:
    df_all.iloc[k,0]=int(df_all.iloc[k,0][5:7])*13
  else:
      df_all.iloc[k,0]=int(df_all.iloc[k,0][5:7])
df_all.rename(columns = {'date':'month_criteria'}, inplace = True)
df_all.head()

## Splitting the dataset into the Training set and Test set

In [ ]:
X = df_all.iloc[:, 1:-1].values
y = df_all.iloc[:, -1].values
X,y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

###import xg boost


In [ ]:

from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train, X_test

## Training the Random Forest Classification model on the Training set

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

## Making the Confusion Matrix for random tree algorithm


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:

y_pred1 = classifier.predict(X_train)
y_pred1
y_pred1df = pd.DataFrame(y_pred1)

In [ ]:
y_pred1df.columns=['crop_type']

## Applying k-Fold Cross Validation



In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

In [ ]:
df_all.to_csv('/content/drive/Shareddrives/arv_msg/colab_notebooks/data_all.csv',index=False)